In [144]:
import skmob
import pandas as pd
from tqdm import tqdm
import plotly.express as px
import geocoder
import datetime

In [13]:
pd.options.display.max_columns = None

## KBR GĄDÓW

In [86]:
df = pd.read_excel('../data/raw/KBR_Gadow_2019_no_pass.xlsx', 'responses', header=[0,1])

In [194]:
# df.head(2)

In [88]:
l = list(df.columns)[199:]
l.append(list(df.columns)[0])
df = df[l]

In [89]:
df.columns = list(map(lambda x: x[0], list(df.columns)))
df = df.rename(columns={'Unnamed: 0_level_0': 'ID odpowiedzi'})

In [193]:
# df.head(2)

In [91]:
new_columns = [
    'user_id',
    'travel_id',
    'start_date',
    'duration_time',
    'mean_of_transport',
    'start_place_type',
    'finish_place_type',
    'start_place',
    'start_region',
    'residence_region',
    'finish_place'
]

travels_df = pd.DataFrame(columns=new_columns)

for i in tqdm(range(1, 28)):
    columns = [
        'ID odpowiedzi',
        f'ID podróży ({i})',
        f'Data rozpoczecia podróży ({i})',
        f'Czas trwania podróży [s] ({i})',
        f'Środek transportu ({i})',
        f'Typ miejsca startu ({i})',
        f'Typ miejsca zakończenia ({i})',
        f'Miejsce startu ({i})',
        f'Numer regionu miejsca startu ({i})',
        f'Numer regionu miejsca zakończenia ({i})',
        f'Miejsce zakończenia ({i})'
    ]

    sub_df = df[columns]
    sub_df = sub_df.dropna()
    sub_df.columns = new_columns

    travels_df = travels_df.append(sub_df)

travels_df['start_date'] = pd.to_datetime(travels_df['start_date'])

100%|██████████| 27/27 [00:00<00:00, 219.91it/s]


In [192]:
# travels_df.head(2)

In [93]:
travels_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 0 to 259
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            1277 non-null   object 
 1   travel_id          1277 non-null   object 
 2   start_date         1277 non-null   object 
 3   duration_time      1277 non-null   object 
 4   mean_of_transport  1277 non-null   object 
 5   start_place_type   1277 non-null   object 
 6   finish_place_type  1277 non-null   object 
 7   start_place        1277 non-null   object 
 8   start_region       1277 non-null   float64
 9   residence_region   1277 non-null   float64
 10  finish_place       1277 non-null   object 
dtypes: float64(2), object(9)
memory usage: 119.7+ KB


In [96]:
plot_df = travels_df[['mean_of_transport', 'travel_id']].groupby('mean_of_transport').count().reset_index(inplace=False).sort_values('travel_id')
fig = px.bar(plot_df, x='mean_of_transport', y='travel_id')
fig.show()

In [155]:
tdf = pd.DataFrame(columns=['travel_id', 'lat', 'long', 'time', 'mean_of_transport'])

for row in tqdm(travels_df.itertuples()):
    g_start = geocoder.osm(row[8])
    g_finish = geocoder.osm(row[11])

    try:
        tdf = tdf.append({
            'travel_id': row[2],
            'lat': g_start.osm['x'],
            'long': g_start.osm['y'],
            'time': row[3],
            'mean_of_transport': row[5]
        }, ignore_index=True)

        tdf = tdf.append({
            'travel_id': row[2],
            'lat': g_finish.osm['x'],
            'long': g_finish.osm['y'],
            'time': row[3] + datetime.timedelta(0,row[4]),
            'mean_of_transport': row[5]
        }, ignore_index=True)
    except Exception as e:
        print()
        print(e)
        print(row[8])
        print(g_start.osm)
        print(row[11])
        print(g_finish.osm)

18it [00:17,  1.00s/it]
'NoneType' object is not subscriptable
Bulwar Ikara, 54-130 Wrocław
{'x': 16.9697476, 'y': 51.1265577, 'addr:street': 'bulwar Ikara', 'addr:city': 'Wrocław', 'addr:state': 'województwo dolnośląskie', 'addr:country': 'Polska', 'addr:postal': '54-130'}
Unnamed Road, Wrocław
None
124it [02:05,  1.01s/it]
'NoneType' object is not subscriptable
Jerzego Bajana 19, 54-129 Wrocław
{'x': 16.9626873, 'y': 51.1283708, 'addr:housenumber': '19', 'addr:street': 'Jerzego Bajana', 'addr:city': 'Wrocław', 'addr:state': 'województwo dolnośląskie', 'addr:country': 'Polska', 'addr:postal': '54-129'}
Braniborska 2/10, 53-600 Wrocław
None
177it [02:58,  1.02s/it]
'NoneType' object is not subscriptable
Stefana Drzewieckiego, Wrocław
{'x': 16.96754, 'y': 51.124409, 'addr:street': 'Stefana Drzewieckiego', 'addr:city': 'Wrocław', 'addr:state': 'województwo dolnośląskie', 'addr:country': 'Polska', 'addr:postal': '54-130'}
Piotra Skargi 1/3, 11-400 Wrocław
None
207it [03:28,  1.01s/it]
'No

In [190]:
# tdf.head()

In [158]:
tdf_copy = tdf.copy()

In [196]:
tdf_copy.to_csv('../data/interim/tdf_gadow.csv')

In [162]:
tdf = skmob.TrajDataFrame(tdf_copy, latitude='long', longitude='lat', datetime='time', user_id='travel_id')

In [191]:
# tdf.head()

In [180]:
len(tdf['uid'].unique())

1024

In [184]:
tdf[tdf['mean_of_transport'] == 'Samochód osobowy jako kierowca'].plot_trajectory(max_users=100)

In [186]:
tdf[tdf['mean_of_transport'] == 'Tramwaj'].plot_trajectory(max_users=100)

In [189]:
tdf[tdf['mean_of_transport'] == 'Komunikacja autobusowa'].plot_trajectory(max_users=100)

In [188]:
tdf[tdf['mean_of_transport'] == 'Pieszo'].plot_trajectory(max_users=100)